# CS246 - Colab 4
## Collaborative Filtering

In [1]:
from IPython.display import Image
print("Colab 4 Mascot")
Image(url='https://media4.giphy.com/media/wtdVYmaRWJ1PyPcc8e/giphy.gif?cid=ecf05e478jdbzo92b89f2b5ud5184xda9cen1xce4bwmjeyu&ep=v1_stickers_search&rid=giphy.gif&ct=s',width=150)

Colab 4 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=a0baffea6e7ae7351af26c13cb46707274bc3d667a747b92813c7c52880c0f01
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:am

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



In [10]:
training.take(3), items.take(3)

([Row(user_id=1, item_id=1, rating=5, timestamp=874965758),
  Row(user_id=1, item_id=2, rating=3, timestamp=876893171),
  Row(user_id=1, item_id=3, rating=4, timestamp=878542960)],
 [Row(item_id=1, movie='Toy Story (1995)'),
  Row(item_id=2, movie='GoldenEye (1995)'),
  Row(item_id=3, movie='Four Rooms (1995)')])

### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [11]:
''' 3 lines of code in total expected.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
print(f"number of rating in train dataset : {training.select('rating').count()}")
print(f"number of rating in test dataset : {test.select('rating').count()}")
print(f"total number of movies : {items.count()}")

number of rating in train dataset : 80000
number of rating in test dataset : 20000
total number of movies : 1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [28]:
''' 5-6 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(maxIter= 5, regParam= 0.01, implicitPrefs= False, userCol= 'user_id', itemCol= 'item_id',
          ratingCol= 'rating', coldStartStrategy= 'drop')
model = als.fit(training)
pred =  model.transform(test)

Now compute the RMSE on the test dataset.


In [29]:
''' 4-5 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
eval = RegressionEvaluator(metricName= 'rmse', labelCol= 'rating',
                           predictionCol= 'prediction')
rmse = eval.evaluate(pred)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 1.1153028899715283


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [22]:
''' 9-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
userRecs = model.recommendForAllUsers(3)
movieRecs = model.recommendForAllItems(3)
userRecs.show(truncate= False), movieRecs.show(truncate= False)

+-------+---------------------------------------------------------+
|user_id|recommendations                                          |
+-------+---------------------------------------------------------+
|1      |[{1160, 6.5482607}, {1093, 6.2612667}, {1195, 6.223744}] |
|2      |[{1643, 9.441108}, {1137, 7.8609443}, {536, 7.7400284}]  |
|3      |[{1278, 9.51662}, {1426, 9.386078}, {1094, 9.369608}]    |
|4      |[{1286, 9.761625}, {1512, 9.690031}, {1127, 9.386446}]   |
|5      |[{1427, 8.561757}, {1113, 7.8201594}, {1184, 7.599305}]  |
|6      |[{1129, 6.3069057}, {1643, 5.983589}, {1512, 5.928462}]  |
|7      |[{1643, 7.137265}, {1022, 6.286331}, {1380, 6.2082057}]  |
|8      |[{906, 9.273066}, {1427, 9.032997}, {1426, 8.994077}]    |
|9      |[{1140, 13.797903}, {962, 11.869198}, {362, 11.394855}]  |
|10     |[{1131, 6.399567}, {1643, 5.976382}, {1368, 5.8514223}]  |
|11     |[{1159, 6.603916}, {1203, 6.1173263}, {1426, 5.9382997}] |
|12     |[{958, 7.9563804}, {1098, 7.4008846}, {

(None, None)

In [24]:
users = test.select(als.getUserCol()).distinct().limit(3)
userSubSetRecs = model.recommendForUserSubset(users, 3)
userSubSetRecs.show(truncate= False)

+-------+--------------------------------------------------------+
|user_id|recommendations                                         |
+-------+--------------------------------------------------------+
|392    |[{793, 6.3921146}, {1643, 6.0786967}, {1368, 5.9607916}]|
|243    |[{512, 5.330564}, {1426, 5.3036795}, {1203, 5.282448}]  |
|148    |[{574, 14.713096}, {1380, 13.220155}, {1022, 12.419528}]|
+-------+--------------------------------------------------------+



In [25]:
movies = test.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 3)
movieSubSetRecs.show(truncate= False)

+-------+---------------------------------------------------+
|item_id|recommendations                                    |
+-------+---------------------------------------------------+
|471    |[{241, 5.368663}, {252, 5.293691}, {811, 5.207682}]|
|463    |[{212, 9.31592}, {9, 8.229258}, {444, 7.414538}]   |
|148    |[{471, 6.233775}, {39, 6.158992}, {589, 6.0845156}]|
+-------+---------------------------------------------------+



Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**!